# Lidar-Camera calibration using the all new python interface!

In this blog post we explain how to perform a lidar-camera calibration with our newly released calibration library. We will calibrate a sensor setup consisting of a Realsense D455 stereo camera and an Ouster OS1 Lidar using our custom design April-Checkerboard.

## Motivation
Autonomous systems need multiple sensors to ensure they can perceive their environment accurately and reliably. Different sensors capture various types of data—like visual, thermal, and spatial information—allowing the system to create a comprehensive understanding of its surroundings. This redundancy helps to overcome the limitations of individual sensors, such as poor performance in certain conditions, and enhances safety, precision, and robustness in decision-making. Multiple sensors working together ensure that the system can adapt to a wide range of scenarios, making it more reliable and effective in real-world applications. 

Good sensor calibration is crucial to ensure that all sensors provide accurate and consistent data, enabling the autonomous system to make precise and reliable decisions. Poor calibration can lead to errors in perception and interpretation, compromising the system's performance and safety.

## Sensor Setup used in this tutorial
The setup involves a RealSense D455 stereo camera and an Ouster OS1 LiDAR, both rigidly mounted on the same mechanical base. Although in this example we use the OS1 you might use any other Ouster Lidar. 

![Sensor Setup](examples/02-camera-lidar/resources/imgs/sensor_setup_small.png)

## Data Requirements
The requirements for the lidar calibration process are the following 

- A rough initial guess of the camera-to-lidar transformation (up to 5°-10°). (Or a board with reflective tape if this doesn't interfere with your lidar point cloud)
- A supported calibration target

## Calibration Target used in this example

For the calibration target we use combination of April Tags and a Checkerboard - that we call April-Checkerboard. However, you may use other supported calibration boards. We refer you to our previous [Blog post](https://www.camcalib.io/post/build-your-own-calibration-tools-and-application-with-the-newest-camcalib-library-release-for-multi)

![Calibration Target](resources/imgs/target_small.png)

## Lidar calibration with different targets
The lidar calibration can consists of two steps.

1. Board plane alignment
2. Intensity alignment (refinement step) - if your lidar provides intensity values.

Step 1 can be performed with every board type our software supports. However, step 2 requires you to use one of the following boards
- April Checkerboard
- Radon Board (requires openCV >=4.3)
- Checkerboard (requires lidar and camera messages to have the same timestamp!)

## Calibration Process
In the following we show the step by step process of the calibration

### Data recording (and best practices)
We use rosbags for the calibration. Ideally, one for the intrinsic camera calibration and one for the lidar-camera extrinsics.
We will skip the intrinsic calibration for this example and simple use the Realsense factory calibration.

### Getting into the code
The example doesn't need anything more than the camcalib module

In [ ]:
import camcalib

Next, we define our calibration target:

In [2]:
target = camcalib.targets.CcMarkerBoard(9, 7, 0.075, 0.1, "A16h5")

Load calibration data, either from folders on the filesystem or like here from a ROS bag:

In [3]:
calib_data = camcalib.calibration.CalibrationData()
calib_data.set_target(target)
with camcalib.utils.ProgressBar() as p:
    calib_data.from_bag('resources/data/2024-08-20-15-27-19_lidar.bag', progress_callback=p.update_progress)

0it [00:00, ?it/s]


----

infra1: found 17328 corners in 459 frames
infra2: found 17376 corners in 459 frames


In [4]:
result = camcalib.calibration.CalibrationResult.load('resources/data/sensors-lidar-init.yaml')

The `sensors-lidar-init.yaml` contains the names of the sensors and also the cameras intrinsics.
We also need to spceify the extrinsics for camera 1 as the identity (`axis_angle: [0, 0, 0]/translation: [0, 0, 0]`) and an initial guess of the extrinsics of the lidar (`axis_angle: [-1.1,  1.1, -1.1]/translation: [0.0, 0.1, 0.0]`).
You can see the file below in the appendix.

## Calibration (Simple)
For best results we recommend performing the lidar calibration at least twice. Once without, and once with using the intensity optimization (`camcalib.CalibrationSettings.use_lidar_intensity_residuals = True`) if you lidar provides reliable intensity data.
The simplest approach looks like the following

**NOTE**: The cell output is unfortunately not in order!

In [14]:
# assumption about the accuracy of the initial guess (should not be higher than 10 [deg])
camcalib.DataSettings.maximum_initial_angle_error = 5
iterations = 2

with camcalib.calibration.CalibratorSession() as cc:
    for i in range(iterations): 
        # unfortunately our test lidar does not have reliable intensity values
        if i > 0:
           camcalib.use_lidar_intensity_residuals = True
           camcalib.DataSettings.maximum_initial_angle_error = 1
        cc.add_calibration_data(calib_data)
        cc.add_calibration(result)
        result, summary = cc.calibrate()
        stats = cc.get_metrics().calculate_reprojection_errors()
        print(stats)
print(result)

[INFO] [2024-08-21T13:31:33Z] [solve]: Final RMSE: 2.201879
[INFO] [2024-08-21T13:31:33Z] [solve]: Solve problem time: 42.203581
[INFO] [2024-08-21T13:31:33Z] [solve]: Success! Solver did converge after 20 iterations.
[INFO] [2024-08-21T13:31:33Z] [calculate_covariances]: Detected 16 cores, using 15.
[INFO] [2024-08-21T13:31:33Z] [calculate_covariances]: Covariance calculation time: 0.000626
[INFO] [2024-08-21T13:31:33Z] [concatenate_extrinsics]: Concatenating the Results with sensor /infra1 as bridge.
[INFO] [2024-08-21T13:31:33Z] [calibrate]: Target extrinsics
[INFO] [2024-08-21T13:31:33Z] [calibrate]: 	(0 <- 0): Pose(r: [0, 0, 0], t: [0, 0, 0])
[INFO] [2024-08-21T13:51:10Z] [initialize_intrinsics]: Available Camera Data: [/infra1, /infra2]
[INFO] [2024-08-21T13:51:10Z] [initialize_intrinsics]: Camera /infra1 already initialized.
[INFO] [2024-08-21T13:51:10Z] [initialize_intrinsics]: Camera /infra2 already initialized.
[INFO] [2024-08-21T13:51:10Z] [initialize_parameters]: Initialize

As we can see, we perform 3 steps.
1. Set the accuracy of the inital guess for the lidar calibration.  
   This basically defines a search region for the lidar target.
   The larger the value, the larger the search region and also the more potential outliers will be in the calibration.
2. Load data (just like for a standard camera calibration)
3. Calibrate without intensities
4. Calibrate again (using the results of the first calibration as initial guess), but this time with intensity optimization.

## Calibration (Advanced and usually more accurate)
As discussed previously we can set the accuracy of the initial guess, which also influences the amount of potential outliers in our data.
Therefore, we recommend to start with a realistic guess and calibrate multiple times, while steadily decreasing this parameter (for less outliers).
Also, we can define an outlier metric to perform an early stopping if the result looks accurate.

**NOTE**: The cell output is unfortunately not in order!

In [13]:
calibration_it_count = 0     # calibration iteration counter
lidar_error_threshold = 0.05  # threshold for recalibration
min_iterations = 4           # perform multiple calibration refining the accuracy each time
max_iterations = 5           # maximum number of iterations for lidar calibration

# assumption about the accuracy of the initial guess (should not be higher than 10 [deg])
camcalib.DataSettings.maximum_initial_angle_error = 5

# This is the condition for the lidar errors to be too high -> recalibrate
lidar_error_check = lambda le: le.total_point_rmse / le.total_point_median > 3 or le.total_point_rmse > lidar_error_threshold

while calibration_it_count < min_iterations or (lidar_errors and lidar_error_check(lidar_errors)):
    if calibration_it_count > 0:
        # after first calibration, use intensities and assume a lidar2cam angular error of max 2°
        camcalib.DataSettings.maximum_initial_angle_error = max_iterations / calibration_it_count
        camcalib.CalibrationSettings.use_lidar_intensity_residuals = True
    with camcalib.calibration.CalibratorSession() as cc:

        # Set data and perform calibration
        cc.add_calibration_data(calib_data)
        cc.add_calibration(result)

        # perform calibration and compute metrics/errors
        result, summary = cc.calibrate()
        metrics = cc.get_metrics()
        lidar_errors = metrics.calculate_lidar_target_errors()
        calibration_it_count += 1

    if calibration_it_count >= max_iterations:
        print(f"Reached max iterations {max_iterations}, stopping!")
        break
        
    if lidar_errors and lidar_error_check(lidar_errors):
        # check if lidar errors are in bounds
        print("Lidar errors too high, recalibrating")
print(result)

[INFO] [2024-08-21T13:26:50Z] [solve]: Final RMSE: 2.201872
[INFO] [2024-08-21T13:26:50Z] [solve]: Solve problem time: 41.449999
[INFO] [2024-08-21T13:26:50Z] [solve]: Success! Solver did converge after 20 iterations.
[INFO] [2024-08-21T13:26:50Z] [calculate_covariances]: Detected 16 cores, using 15.
[INFO] [2024-08-21T13:26:50Z] [calculate_covariances]: Covariance calculation time: 0.000524
[INFO] [2024-08-21T13:26:50Z] [concatenate_extrinsics]: Concatenating the Results with sensor /infra1 as bridge.
[INFO] [2024-08-21T13:26:50Z] [calibrate]: Target extrinsics
[INFO] [2024-08-21T13:26:50Z] [calibrate]: 	(0 <- 0): Pose(r: [0, 0, 0], t: [0, 0, 0])
[INFO] [2024-08-21T13:28:33Z] [initialize_intrinsics]: Available Camera Data: [/infra1, /infra2]
[INFO] [2024-08-21T13:28:33Z] [initialize_intrinsics]: Camera /infra1 already initialized.
[INFO] [2024-08-21T13:28:33Z] [initialize_intrinsics]: Camera /infra2 already initialized.
[INFO] [2024-08-21T13:28:33Z] [initialize_parameters]: Initialize

If you like, you can now save the calibration as a yaml file

In [12]:
result.save("resources/data/sensors-lidar-result.yaml")

## Visualization of the result
Looking at the results as coordinate systems we can see, that it matches the physical setup above.

![Result Coordinate Systems](resources/imgs/calib_cosys.png)

## Visualization of Lidar points on Target
Here, we want to show a before and after illustration of the lidar points on the camera target.
This shows the initial miss-alignment of the initial guess vs. the calibrated result.

![Point Comparison](resources/imgs/lidar_large_before_n_after.png)

# Comparison of initial guess vs results
Comparing the initial guess with the actual result gives us

### Initial Guess
`axis_angle: [-1.1,  1.1, -1.1]`  
`translation: [0.0, 0.1, 0.0]`  
### Actual Calibration
`axis_angle: [-1.162643609301952, 1.2383899554908, -1.156430220577199]`  
`translation: [0.08961128552674028, 0.227704752288137, -0.1212762302770224]`

### Difference
This is an approximate difference in rotation of **9°** and a translation of around **20cm**
These values are also typically the boundaries we suggest for the inaccuracy for the initial guess of the lidar pose.

# Conclusion
With this tutorial you should now be able to use our new calibration library to perform a camera to lidar calibration. You should know the capabilities and requirements to do so, and also the correct workflow.
If you have any questions about the software or the calibration process, feel free to contact us.

# Appendix
## sensors-lidar-init.yaml

In [ ]:
sensors:
  camera1/infra1/image_rect_raw:
    extrinsics:
      axis_angle: [0, 0, 0]
      translation: [0, 0, 0]
    intrinsics:
      parameters:
        fx: 655.714
        fy: 656.328
        cx: 647.291
        cy: 398.318
        image_size:
        - 1280
        - 800
      type: Pinhole
    extrinsics:
      axis_angle: [0, 0, 0]
      translation: [0, 0, 0]
  camera1/infra2/image_rect_raw:
    intrinsics:
      parameters:
        fx: 655.555
        fy: 656.181
        cx: 648.16
        cy: 398.849
        image_size:
        - 1280
        - 800
      type: Pinhole
  ouster/points:
    extrinsics: 
      axis_angle: [-1.1,  1.1, -1.1]
      translation: [0.0, 0.1, 0.0]
    intrinsics:
      type: LIDAR

## sensors-lidar-result.yaml

In [ ]:
sensors:
  camera1/infra1/image_rect_raw:
    type: CAMERA
    intrinsics:
      type: Pinhole
      parameters:
        fx: 655.7140000000001
        fy: 656.328
        cx: 647.2910000000001
        cy: 398.318
        image_size:
          - 1280
          - 800
    extrinsics:
      axis_angle: [0, 0, 0]
      translation: [0, 0, 0]
  camera1/infra2/image_rect_raw:
    type: CAMERA
    intrinsics:
      type: Pinhole
      parameters:
        fx: 655.5549999999999
        fy: 656.181
        cx: 648.16
        cy: 398.849
        image_size:
          - 1280
          - 800
    extrinsics:
      axis_angle: [8.670206090169928e-05, 0.0004850419829581639, 0.0001061361933542322]
      translation: [-0.09503610933865565, 0.0001459713103361074, -6.55041539028914e-05]
  ouster/points:
    type: LIDAR
    extrinsics:
      axis_angle: [-1.162643609301952, 1.2383899554908, -1.156430220577199]
      translation: [0.08961128552674028, 0.227704752288137, -0.1212762302770224]